<a href="https://colab.research.google.com/github/AnshuKamath/DB-Analytics-Assignment/blob/main/Section1_R_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Section 1: Data Analysis in R - Analyzing the Renewing Innovation:
# GreenFuture's Transformation through Technology

In [ ]:
!apt-get install -y r-base

In [ ]:
# Install required R packages
!R -e "install.packages(c('RSQLite', 'DBI', 'dplyr', 'ggplot2'), repos='https://cran.rstudio.com/')"

In [ ]:
# Load the R extension
%load_ext rpy2.ipython

In [ ]:
# Clone your GitHub repository to access the dataset
!git clone https://github.com/AnshuKamath/DB-Analytics-Assignment.git

In [3]:
# Importing dataset to Google Colab
github_url <- "https://raw.githubusercontent.com/AnshuKamath/DB-Analytics-Assignment/main/greenfuture_ideas_dataset.csv"

In [6]:
# Download the dataset from GitHub
greenfuture_data <- read.csv("https://raw.githubusercontent.com/AnshuKamath/DB-Analytics-Assignment/main/greenfuture_ideas_dataset.csv")

In [ ]:
# Display the first few rows to confirm successful import
head(greenfuture_data)
str(greenfuture_data)
summary(greenfuture_data)

In [13]:
# Part 2A: Demonstrating SQL Operations in R (Fixed Column Names)
# This script demonstrates basic SQL operations in R

# Load required libraries
library(RSQLite)
library(DBI)
library(dplyr)

# Create a connection to a new SQLite database
conn <- dbConnect(RSQLite::SQLite(), ":memory:")

# Create a table and import the dataset
# Note: This assumes greenfuture_data is already loaded from Part 1
dbWriteTable(conn, "ideas", greenfuture_data)

# Confirm that the table was created successfully
dbListTables(conn)

# Check the column names to ensure we use the correct ones
column_names <- dbGetQuery(conn, "PRAGMA table_info(ideas)")
print("Column names in the database:")
print(column_names$name)

# 1. SELECT Operation - Basic retrieval
select_query <- "SELECT * FROM ideas LIMIT 5"
result1 <- dbGetQuery(conn, select_query)
print("Basic SELECT operation:")
print(result1)

# 2. SELECT Operation with filtering - Get ideas with high votes
high_votes_query <- "SELECT \"Employee.ID\", \"Idea.Title\", \"Number.of.Votes\"
                   FROM ideas
                   WHERE \"Number.of.Votes\" > 50
                   ORDER BY \"Number.of.Votes\" DESC
                   LIMIT 10"
result2 <- dbGetQuery(conn, high_votes_query)
print("SELECT with filtering - High votes:")
print(result2)

# 3. SELECT with conditional filtering - Ideas from specific locations
location_query <- "SELECT * FROM ideas WHERE \"Office.Location\" IN ('London', 'Tokyo', 'New York') LIMIT 5"
result3 <- dbGetQuery(conn, location_query)
print("SELECT with location filtering:")
print(result3)

# 4. INSERT Operation - Add a new idea record
# Check if there are any existing records with Employee.ID 'EMP3001'
check_existing <- dbGetQuery(conn, "SELECT COUNT(*) as count FROM ideas WHERE \"Employee.ID\" = 'EMP3001'")
if (check_existing$count == 0) {
  insert_query <- "INSERT INTO ideas
                  (\"Employee.ID\", \"Office.Location\", Department, \"Idea.Submission.Date\", \"Idea.Title\",
                   \"Idea.Category\", \"Number.of.Votes\", \"Collaboration.Status\", \"Approval.Status\",
                   \"Implementation.Status\", \"Security.Concern.Flag\")
                  VALUES
                  ('EMP3001', 'London', 'Research', '2023-05-01', 'AI-Driven Carbon Footprint Analyzer',
                   'Renewable Energy', 0, 'Not Started', 'Pending', 'Not Started', 'No')"
  dbExecute(conn, insert_query)
  print("New idea inserted successfully")
} else {
  print("Record with EMP3001 already exists, skipping insertion")
}

# Verify the insertion
verify_insert <- "SELECT * FROM ideas WHERE \"Employee.ID\" = 'EMP3001'"
result4 <- dbGetQuery(conn, verify_insert)
print("Verifying INSERT operation:")
print(result4)

# 5. UPDATE Operation - Update the number of votes for the new idea
update_query <- "UPDATE ideas
                SET \"Number.of.Votes\" = 15, \"Collaboration.Status\" = 'In Progress'
                WHERE \"Employee.ID\" = 'EMP3001'"
rows_affected <- dbExecute(conn, update_query)
print(paste("Rows updated:", rows_affected))

# Verify the update
verify_update <- "SELECT * FROM ideas WHERE \"Employee.ID\" = 'EMP3001'"
result5 <- dbGetQuery(conn, verify_update)
print("Verifying UPDATE operation:")
print(result5)

# 6. DELETE Operation - Delete the newly added record
delete_query <- "DELETE FROM ideas WHERE \"Employee.ID\" = 'EMP3001'"
rows_deleted <- dbExecute(conn, delete_query)
print(paste("Rows deleted:", rows_deleted))

# Verify the deletion
verify_delete <- "SELECT * FROM ideas WHERE \"Employee.ID\" = 'EMP3001'"
result6 <- dbGetQuery(conn, verify_delete)
if (nrow(result6) == 0) {
  print("Record successfully deleted")
} else {
  print("Record still exists - deletion failed")
}

# Note: We're not disconnecting from the database here because we'll use
# the same connection in the next part

[1] "ideas"

[1] "Column names in the database:"
 [1] "Employee.ID"           "Office.Location"       "Department"           
 [4] "Idea.Submission.Date"  "Idea.Title"            "Idea.Category"        
 [7] "Number.of.Votes"       "Collaboration.Status"  "Approval.Status"      
[10] "Implementation.Status" "Security.Concern.Flag"
[1] "Basic SELECT operation:"
  Employee.ID Office.Location           Department Idea.Submission.Date
1    587b45d8         Germany     Renewable Energy           16/04/2024
2    0d3e8b8e          Canada     Renewable Energy           14/01/2024
3    fa012fe7          France Environmental Policy           10/01/2025
4    cb8ed680           Japan           Technology           24/12/2023
5    333fff26       Australia           Technology           13/02/2025
  Idea.Title         Idea.Category Number.of.Votes Collaboration.Status
1     Idea 1 Sustainable Transport             231   Single Contributor
2     Idea 2      Water Management             477  Cross-Regional Team
3 

In [14]:
# Part 2B: Applying Mathematical and Aggregate Functions (Fixed Column Names)
# This script demonstrates SQL aggregate and mathematical functions in R

# 1. Count total number of ideas by category with percentage
category_count_query <- "
SELECT
    \"Idea.Category\",
    COUNT(*) as idea_count,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM ideas), 2) as percentage
FROM
    ideas
GROUP BY
    \"Idea.Category\"
ORDER BY
    idea_count DESC"
category_counts <- dbGetQuery(conn, category_count_query)
print("1. Ideas by category with percentage:")
print(category_counts)

# 2. Calculate average, maximum, and minimum votes by office location
votes_by_location_query <- "
SELECT
    \"Office.Location\",
    COUNT(*) as total_ideas,
    ROUND(AVG(\"Number.of.Votes\"), 2) as average_votes,
    MAX(\"Number.of.Votes\") as max_votes,
    MIN(\"Number.of.Votes\") as min_votes
FROM
    ideas
GROUP BY
    \"Office.Location\"
ORDER BY
    average_votes DESC"
votes_by_location <- dbGetQuery(conn, votes_by_location_query)
print("2. Vote statistics by office location:")
print(votes_by_location)

# 3. Calculate approval rate by department using CASE statements
approval_rate_query <- "
SELECT
    Department,
    COUNT(*) as total_ideas,
    SUM(CASE WHEN \"Approval.Status\" = 'Approved' THEN 1 ELSE 0 END) as approved_ideas,
    ROUND(100.0 * SUM(CASE WHEN \"Approval.Status\" = 'Approved' THEN 1 ELSE 0 END) / COUNT(*), 2) as approval_rate
FROM
    ideas
GROUP BY
    Department
ORDER BY
    approval_rate DESC"
approval_rates <- dbGetQuery(conn, approval_rate_query)
print("3. Approval rates by department:")
print(approval_rates)

# 4. Analyze security concerns by category using mathematical calculations
security_query <- "
SELECT
    \"Idea.Category\",
    COUNT(*) as total_ideas,
    SUM(CASE WHEN \"Security.Concern.Flag\" = 'Yes' THEN 1 ELSE 0 END) as security_concerns,
    ROUND(100.0 * SUM(CASE WHEN \"Security.Concern.Flag\" = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) as security_concern_rate
FROM
    ideas
GROUP BY
    \"Idea.Category\"
ORDER BY
    security_concern_rate DESC"
security_analysis <- dbGetQuery(conn, security_query)
print("4. Security concerns by category:")
print(security_analysis)

# 5. Using date functions to analyze submission patterns
date_analysis_query <- "
SELECT
    strftime('%Y-%m', \"Idea.Submission.Date\") as month_year,
    COUNT(*) as submissions,
    ROUND(AVG(\"Number.of.Votes\"), 2) as avg_votes,
    SUM(CASE WHEN \"Approval.Status\" = 'Approved' THEN 1 ELSE 0 END) as approvals,
    ROUND(100.0 * SUM(CASE WHEN \"Approval.Status\" = 'Approved' THEN 1 ELSE 0 END) / COUNT(*), 2) as approval_rate
FROM
    ideas
GROUP BY
    month_year
ORDER BY
    month_year"
date_analysis <- dbGetQuery(conn, date_analysis_query)
print("5. Monthly submission patterns:")
print(date_analysis)

# 6. Using subqueries to compare against global averages
relative_performance_query <- "
SELECT
    Department,
    ROUND(AVG(\"Number.of.Votes\"), 2) as dept_avg_votes,
    (SELECT ROUND(AVG(\"Number.of.Votes\"), 2) FROM ideas) as global_avg_votes,
    ROUND(AVG(\"Number.of.Votes\") - (SELECT AVG(\"Number.of.Votes\") FROM ideas), 2) as vote_difference,
    CASE
        WHEN AVG(\"Number.of.Votes\") > (SELECT AVG(\"Number.of.Votes\") FROM ideas) THEN 'Above Average'
        ELSE 'Below Average'
    END as performance
FROM
    ideas
GROUP BY
    Department
ORDER BY
    dept_avg_votes DESC"
relative_performance <- dbGetQuery(conn, relative_performance_query)
print("6. Department performance relative to global average:")
print(relative_performance)

# 7. Using HAVING to filter aggregated results
high_approval_locations_query <- "
SELECT
    \"Office.Location\",
    COUNT(*) as total_ideas,
    SUM(CASE WHEN \"Approval.Status\" = 'Approved' THEN 1 ELSE 0 END) as approved_ideas,
    ROUND(100.0 * SUM(CASE WHEN \"Approval.Status\" = 'Approved' THEN 1 ELSE 0 END) / COUNT(*), 2) as approval_rate
FROM
    ideas
GROUP BY
    \"Office.Location\"
HAVING
    approval_rate > 50 AND total_ideas >= 10
ORDER BY
    approval_rate DESC"
high_approval_locations <- dbGetQuery(conn, high_approval_locations_query)
print("7. High-approval office locations (>50% approval, ≥10 ideas):")
print(high_approval_locations)

# 8. Using mathematical functions to analyze collaboration impact
collaboration_impact_query <- "
SELECT
    \"Collaboration.Status\",
    COUNT(*) as idea_count,
    ROUND(AVG(\"Number.of.Votes\"), 2) as avg_votes,
    SUM(CASE WHEN \"Approval.Status\" = 'Approved' THEN 1 ELSE 0 END) as approved_count,
    ROUND(100.0 * SUM(CASE WHEN \"Approval.Status\" = 'Approved' THEN 1 ELSE 0 END) / COUNT(*), 2) as approval_rate,
    ROUND(
        100.0 * SUM(CASE WHEN \"Approval.Status\" = 'Approved' THEN 1 ELSE 0 END) / COUNT(*) /
        (SELECT 100.0 * SUM(CASE WHEN \"Approval.Status\" = 'Approved' THEN 1 ELSE 0 END) / COUNT(*) FROM ideas),
        2
    ) as relative_approval_effectiveness
FROM
    ideas
GROUP BY
    \"Collaboration.Status\"
ORDER BY
    approval_rate DESC"
collaboration_impact <- dbGetQuery(conn, collaboration_impact_query)
print("8. Collaboration impact analysis:")
print(collaboration_impact)

# Now we can close the database connection as we're done with SQL operations
dbDisconnect(conn)
print("Database connection closed")

[1] "1. Ideas by category with percentage:"
          Idea.Category idea_count percentage
1      Renewable Energy        533      17.77
2 Sustainable Transport        512      17.07
3      Circular Economy        496      16.53
4      Water Management        492      16.40
5          Smart Cities        489      16.30
6        Carbon Capture        478      15.93
[1] "2. Vote statistics by office location:"
   Office.Location total_ideas average_votes max_votes min_votes
1           Mexico         240        264.75       498         1
2            Japan         259        263.24       497         1
3           Brazil         227        254.92       500         4
4        Australia         244        254.52       499         1
5              USA         241        250.78       500         1
6            China         280        248.97       500         0
7           Canada         252        247.01       499         0
8            India         222        246.60       499         1
9   